## Loading Data

In [26]:
import pandas as pd
import datetime

In [19]:
filename = "./../data/311_Cases.csv"

In [39]:
data = pd.read_csv(filename, encoding='utf8',engine='python').dropna()

In [41]:
# quick inspection
len(data) # 3,362,678
data.head()

3362678

## Downsize data

In [44]:
small_data = data.sample(n = int(len(data) * 0.1)) # keep 10%
len(small_data) # 336,267

336267

## Select relevant variables

In [112]:
# select relevant variables
# goal: predict how long it takes to close a case, given neighborhood and category of complaint
# (might want to move Category to Request Type?)

X = small_data[["Latitude", "Longitude", "Category"]] 

timedelta = pd.to_datetime(small_data["Closed"]) - pd.to_datetime(small_data["Opened"])

In [113]:
small_data["ClosingTime"] = timedelta.dt.days

In [116]:
small_data.head()

53928

# Test Train Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


In [ ]:
clf_2 = KNeighborsClassifier(n_neighbors=5)
clf_2.fit(X_train,y_train)